# Exercise 15: Power Analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below. 

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill. 

`c` is reading comprehension, and `x` is word reading experience.

In [ ]:
sample_size = 100 # How many children in data set? 
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0. 
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
      age <- runif(sample_size, age_lo, age_hi)
      x <- (beta_xa * age) + beta_x0 + rnorm(sample_size, mean = 0, sd = sd_x)
      c <- (beta_ca * age) + (beta_cx * x) + beta_c0 + rnorm(sample_size, mean = 0, sd = sd_c)
      return(data.frame(age=age,x=x,c=c)) # it's actually bad form to have a variable named "c" in R, my bad...
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,x,c
,<dbl>,<dbl>,<dbl>
1,169.5380,142.59922,520.8732
2,141.5639,116.18801,396.7923
3,139.6971,81.23672,381.5143
4,173.8004,91.40009,498.1683
5,188.0922,133.33126,525.2732
6,160.3735,45.57972,223.9429


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data. 

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously. 

In [ ]:
# WRITE YOUR CODE HERE
#install.packages('mediation')
library(mediation)

run_analysis <- function(simulated_data){
  fitM <- lm(x ~ age, data=simulated_data) 
  fitY <- lm(c ~ x + age, data=simulated_data) 
  fit_mediate <- mediate(fitM, fitY, treat = 'age', mediator = 'x')
  return(c(fit_mediate$d0, fit_mediate$d0.p, fit_mediate$z0, fit_mediate$z0.p))
}

# run on previously simulated data and print
run_analysis(dat)

[1] 1.236438 0.004000 1.062073 0.000000

---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [ ]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)
    
    # Start simulating
    for (i in 1:num_simulations) {
      # simulate data
      dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, 
                           sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      
      # fit mediation model and save effect estimates and store
      simouts[i,] <- run_analysis(dat) 
    }
    
    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov = mean(simouts[,2] <= alpha)
    ADE_cov =  mean(simouts[,4] <= alpha)

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01. 

In [ ]:
# WRITE YOUR CODE HERE
coverages <- repeat_analysis(num_simulations = 10, alpha = 0.01, sample_size, 
      age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

# print ACME and ADE coverages
coverages

$ACME_cov
[1] 0.9

$ADE_cov
[1] 0.5

---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run. 

In [ ]:
# WRITE YOUR CODE HERE
library(tidyverse)

dat <- expand.grid(samps = c(50,75,100,125,150)) 

res <- dat %>%
  mutate(ACME_COV = map(samps, ~ repeat_analysis(num_simulations = 10, 
  alpha = 0.01, sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, 
  beta_cx, beta_c0, sd_c)$ACME_cov)) %>% # ACME Coverage
  mutate(ADE_COV = map(samps, ~ repeat_analysis(num_simulations = 10, 
  alpha = 0.01, sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, 
  beta_cx, beta_c0, sd_c)$ADE_cov)) # ADE Coverage

# Note: I am unsure how to avoid making two calls to map() here, which I know is
# not particularly efficient. I think map2() allows for parallel looping, but I
# think its possible I just don't understand how to assign list output from a 
# mapped function to new columns in a dataframe. 

Print your results. 

In [ ]:
res

samps,ACME_COV,ADE_COV
<dbl>,<list>,<list>
50,0.5,0.3
75,0.7,0.9
100,0.5,0.7
125,0.9,0.8
150,0.9,0.9


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above? 

> Based on the results of the simulations, we acheive maximum coverage of both the indirect and direct effects for an alpha level of .01 at a sample size of 150. 
> 

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> Given that c was generated using information from both x and age, the mediated effect is more present in the simulated data than the direct effect of age on c. As a result, the direct effect was harder to detect in the MC simulation than the mediated effect.

**DUE:** 5pm EST, April 5, 2023

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*